In [13]:
import mediapipe as mp
import pickle as pk
from tqdm import tqdm
import cv2
import os
import pickle
import datetime as dt

### Creating labelled dataset

In [9]:
# initiating mediapipe configurations

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

In [10]:
# initializing image directory

DATA_DIR = './data/gen_data'
data_prefix = DATA_DIR.split('/')[-1]

In [11]:
data = []
labels = []
max_count = 20

for dir_ in tqdm(os.listdir(DATA_DIR)):
    if dir_ == '.DS_Store':
        continue
    count = 0
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []
        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x)
                    data_aux.append(y)

            data.append(data_aux)
            labels.append(dir_)
        
        count += 1
        if count == max_count:
            break

100%|██████████| 4/4 [00:01<00:00,  2.17it/s]


In [14]:
now = dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
f = open(f'prep_data/{data_prefix}_{now}.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()